In [24]:
# =============================================================================
# CÉLULA 1: IMPORTAÇÕES E CONFIGURAÇÃO INICIAL
# =============================================================================

# --- Importação das Bibliotecas Essenciais ---
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import seaborn as sns
from fpdf import FPDF
import re
from PIL import Image
import squarify
from math import pi
from datetime import datetime

print("Bibliotecas importadas com sucesso.")

# --- Definição dos Caminhos (Paths) ---
# Usando os caminhos especificados pelo usuário
output_dir = '../../analises/'
temp_img_dir = os.path.join(output_dir, 'temp_images/')

# --- Criação dos Diretórios de Saída ---
# Garante que as pastas para salvar o relatório e as imagens existam
os.makedirs(output_dir, exist_ok=True)
# Diretório de saída modificado
if not os.path.exists(temp_img_dir):
    os.makedirs(temp_img_dir)

print(f"Diretório de saída configurado em: {os.path.abspath(output_dir)}")
print(f"Diretório de imagens temporárias configurado em: {os.path.abspath(temp_img_dir)}")



Bibliotecas importadas com sucesso.
Diretório de saída configurado em: c:\Users\pedro\OneDrive\Documentos\GitHub\Repositorio_pesquisas\analises_vitimizacao\analises
Diretório de imagens temporárias configurado em: c:\Users\pedro\OneDrive\Documentos\GitHub\Repositorio_pesquisas\analises_vitimizacao\analises\temp_images


In [25]:
# Função adaptada para ler os dados usando o arquivo de mapa
def carregar_dados_mapeados(caminhos_csv):
    """
    Carrega arquivos CSV que foram salvos com nomes de hash, utilizando um
    arquivo '_mapa_de_nomes.csv' para restaurar os nomes originais como chaves
    do dicionário.
    """
    dados = {}
    
    # Se a lista de caminhos estiver vazia, não faz nada
    if not caminhos_csv:
        print("[AVISO] Nenhuma arquivo CSV encontrado para carregar.")
        return dados

    # Pega o diretório da categoria a partir do primeiro arquivo da lista
    dir_categoria = os.path.dirname(caminhos_csv[0])
    caminho_mapa = os.path.join(dir_categoria, '_mapa_de_nomes.csv')

    # 1. Tenta carregar o arquivo de mapeamento
    try:
        print(f"\n--- Carregando dados da categoria em '{dir_categoria}' ---")
        mapa_df = pd.read_csv(caminho_mapa, sep=';', encoding='utf-8')
        print(f"[INFO] Arquivo de mapa '{caminho_mapa}' carregado com sucesso.")
    except FileNotFoundError:
        print(f"[ERRO] Arquivo de mapa '_mapa_de_nomes.csv' não encontrado em '{dir_categoria}'.")
        print("       Não é possível carregar os dados sem o mapa. Pulando esta categoria.")
        return dados
    except Exception as e:
        print(f"[ERRO] Falha ao ler o arquivo de mapa '{caminho_mapa}': {e}")
        return dados

    # 2. Itera sobre o mapa para carregar cada arquivo de dados
    for index, linha in mapa_df.iterrows():
        nome_original = linha['nome_original']
        nome_hash = linha['nome_arquivo_hash']
        
        caminho_dado = os.path.join(dir_categoria, f"{nome_hash}.csv")
        
        try:
            df = pd.read_csv(
                caminho_dado,
                sep=';',
                encoding='utf-8',
                header=0,
                index_col=0,
            )
            # Usa o NOME ORIGINAL como chave do dicionário
            dados[nome_original] = df
            print(f"  [OK] CSV para '{nome_original}' carregado com sucesso.")
        
        except FileNotFoundError:
            print(f"  [ERRO] Arquivo de dados '{caminho_dado}' não encontrado, mas listado no mapa.")
        except Exception as e:
            print(f"  [ERRO] Ao ler o arquivo de dados '{caminho_dado}': {e}")
            
    return dados

# --- SCRIPT PRINCIPAL DE CARREGAMENTO ---

# 1. Encontra todos os arquivos .csv como antes
caminho_pasta = '../../dados_tratados/PNAD_2009/csv'

agressao_paths = glob.glob(os.path.join(caminho_pasta, 'agressao/*.csv'), recursive=True)
furto_paths = glob.glob(os.path.join(caminho_pasta, 'furto/*.csv'), recursive=True)
roubo_paths = glob.glob(os.path.join(caminho_pasta, 'roubo/*.csv'), recursive=True)
roubofurto_paths = glob.glob(os.path.join(caminho_pasta, 'roubofurto/*.csv'), recursive=True)
seguranca_paths = glob.glob(os.path.join(caminho_pasta, 'seguranca/*.csv'), recursive=True)
tentativa_paths = glob.glob(os.path.join(caminho_pasta, 'tentativa/*.csv'), recursive=True)

# 2. Usa a NOVA função para carregar os dados
dados_carregados = {
    "agressao": carregar_dados_mapeados(agressao_paths),
    "furto": carregar_dados_mapeados(furto_paths),
    "roubo": carregar_dados_mapeados(roubo_paths),
    "roubofurto": carregar_dados_mapeados(roubofurto_paths),
    "seguranca": carregar_dados_mapeados(seguranca_paths),
    "tentativa": carregar_dados_mapeados(tentativa_paths)
}

print("\n\n[SUCESSO] Processo de carregamento de dados concluído.")

# 3. Verificação (opcional, mas recomendado)
# Vamos verificar as chaves carregadas para a categoria 'agressao'
print("\nVerificando as chaves (nomes dos DataFrames) carregadas para a categoria 'agressao':")
if dados_carregados['agressao']:
    print(list(dados_carregados['agressao'].keys()))
else:
    print("Nenhum dado carregado para 'agressao'.")

"""
Gere visualizações de dados a partir dos arquivos CSV anexados sobre violência (`agressao.xlsx - *.csv`).

Cada arquivo é uma tabela onde a primeira coluna é a categoria principal. Priorize a análise e criação de gráficos que explorem as relações entre:

- Tipos de **agressão** (física, psicológica, etc.).
- Comparações por **sexo** (colunas 'Homens' e 'Mulheres').
- Distribuição por **cor/raça** (colunas 'Branca', 'Preta', 'Parda', etc.).
- Recortes por **faixa etária** ou **ciclo de vida**.
- O **local da ocorrência** da agressão.

Crie gráficos de barras, de pizza e comparativos que revelem os padrões mais significativos nesses cruzamentos de dados.
"""


--- Carregando dados da categoria em '../../dados_tratados/PNAD_2009/csv\agressao' ---
[INFO] Arquivo de mapa '../../dados_tratados/PNAD_2009/csv\agressao\_mapa_de_nomes.csv' carregado com sucesso.
  [OK] CSV para 'Coeficientes de variação das estimativas de pessoas de 10 anos ou mais de idade que foram vítimas de agressão física, no período de referência de 365 dias, por sexo e cor ou raça, segundo a situação do domicílio, as Grandes Regiões e as Unidades da Federação' carregado com sucesso.
  [OK] CSV para 'Coeficientes de variação das estimativas de distribuição das pessoas de 10 anos ou mais de idade que foram vítimas de agressão física, no período de referência de 365 dias, por sexo e cor ou raça, segundo a situação do domicílio, as Grandes Regiões e as Unidades da Federação' carregado com sucesso.
  [OK] CSV para 'Coeficientes de variação das estimativas de percentual de pessoas que foram vítimas de agressão física, no período de referência de 365 dias, na população de 10 anos o

"\nGere visualizações de dados a partir dos arquivos CSV anexados sobre violência (`agressao.xlsx - *.csv`).\n\nCada arquivo é uma tabela onde a primeira coluna é a categoria principal. Priorize a análise e criação de gráficos que explorem as relações entre:\n\n- Tipos de **agressão** (física, psicológica, etc.).\n- Comparações por **sexo** (colunas 'Homens' e 'Mulheres').\n- Distribuição por **cor/raça** (colunas 'Branca', 'Preta', 'Parda', etc.).\n- Recortes por **faixa etária** ou **ciclo de vida**.\n- O **local da ocorrência** da agressão.\n\nCrie gráficos de barras, de pizza e comparativos que revelem os padrões mais significativos nesses cruzamentos de dados.\n"

In [26]:
display(dados_carregados['agressao']['Coeficiente de variação das estimativas de pessoas de 10 anos ou mais de idade que foram vítimas de agressão física, no período de referência de 365 dias, por sexo e cor ou raça, segundo algumas Unidades da Federação e Regiões Metropolitanas'])

,total,homens,mulheres,branca,preta/parda
pará,7.58,7.90,11.18,15.12,7.49
região metropolitana de belém,8.57,10.07,11.52,16.38,9.57
ceará,8.67,8.19,13.53,14.51,8.98
região metropolitana de fortaleza,7.83,9.27,11.40,13.59,8.95
pernambuco,6.58,7.96,9.18,11.82,7.91
região metropolitana de recife,8.64,11.01,12.52,12.90,11.22
bahia,6.10,7.01,8.32,15.22,5.97
região metropolitana de salvador,6.23,7.76,9.18,13.71,6.63
minas gerais,7.32,8.49,8.57,11.86,7.64
região metropolitana de belo horizonte,9.10,11.38,12.17,17.64,10.36


In [27]:
# --- ETAPA 1: CONFIGURAÇÃO E GERAÇÃO DOS GRÁFICOS ---

print("\n--- Iniciando geração sequencial de gráficos ---")

# Criando uma pasta para salvar os gráficos.
output_folder = 'graficos'
os.makedirs(output_folder, exist_ok=True)
print(f"Pasta '{output_folder}' pronta para receber os gráficos.")

# Dicionário para simular os dados carregados
dados_carregados = {
    'agressao': {
        'Coeficientes de variação das estimativas de pessoas de 10 anos ou mais de idade que foram vítimas de agressão física, no período de referência de 365 dias, por sexo e cor ou raça, segundo a situação do domicílio, as Grandes Regiões e as Unidades da Federação': pd.DataFrame(np.random.rand(10, 5) * 20, columns=['total', 'homens', 'mulheres', 'branca', 'preta/parda']),
        'Coeficientes de variação das estimativas de pessoas de 10 anos ou mais de idade que foram vítimas de agressão física, no período de referência de 365 dias, por grupos de idade, segundo a situação do domicílio e as Grandes Regiões': pd.DataFrame(np.random.rand(10, 6) * 20, columns=['Total', '10 a 15 anos', '16 a 24 anos', '25 a 34 anos', '35 a 49 anos', '50 a 59 anos (ou mais)']),
        'Coeficientes de variação das estimativas de pessoas de 10 anos ou mais de idade que não procuraram a polícia em decorrência da última agressão física que foram vítimas, no período de referência de 365 dias, por motivo de não terem procurado a polícia em decorrência da última agressão física, segundo algumas Unidades da Federação e Regiões Metropolitanas': pd.DataFrame(np.random.rand(10, 6), columns=['total', 'falta de provas', 'nao era importante', 'nao acreditavam na policia', 'nao queriam envolver a policia ou medo de represalias', 'outro'])
    },
    'furto': {
        'Coeficientes de variação das estimativas de percentual de pessoas que foram vítimas de furto, no período de referência de 365 dias, na população de 10 anos ou mais de idade, por sexo e cor ou raça, segundo a situação do domicílio, as Grandes Regiões e as Unidades da Federação': pd.DataFrame(np.random.rand(10, 5) * 20, columns=['Total', 'Homens', 'Mulheres', 'Branca', 'Preta/Parda']),
        'Coeficientes de variação das estimativas de pessoas de 10 anos ou mais de idade que foram vítimas de furto, no período de referência de 365 dias, por grupos de idade, segundo a situação do domicílio, as Grandes Regiões e as Unidades da Federação': pd.DataFrame(np.random.rand(10, 6) * 20, columns=['Total', '10 a 15 anos', '16 a 24 anos', '25 a 34 anos', '35 a 49 anos', '50 a 59 anos (ou mais)']),
        'Coeficientes de variação das estimativas de pessoas de 10 anos ou mais de idade que não procuraram a polícia em decorrência do último furto que foram vítimas, no período de referência de 365 dias, por motivo de não terem procurado a polícia em decorrência do último furto, segundo a situação do domicílio,': pd.DataFrame(np.random.rand(10, 6), columns=['total', 'falta de provas', 'nao era importante', 'nao acreditavam na policia', 'nao queriam envolver a policia ou medo de represalias', 'outro'])
    },
    'roubo': {
        'Coeficientes de variação das estimativas de percentual de pessoas que foram vítimas de roubo, no período de referência de 365 dias, na população de 10 anos ou mais de idade, por sexo e cor ou raça, segundo a situação do domicílio, as Grandes Regiões e as Unidades da Federação': pd.DataFrame(np.random.rand(10, 5) * 20, columns=['Total', 'Homens', 'Mulheres', 'Branca', 'Preta/Parda']),
        'Coeficientes de variação das estimativas de pessoas de 10 anos ou mais de idade que foram vítimas de roubo, no período de referência de 365 dias, por grupos de idade, segundo a situação do domicílio, as Grandes Regiões e as Unidades da Federação': pd.DataFrame(np.random.rand(10, 6) * 20, columns=['Total', '10 a 15 anos', '16 a 24 anos', '25 a 34 anos', '35 a 49 anos', '50 a 59 anos (ou mais)']),
        'Coeficientes de variação das estimativas de pessoas de 10 anos ou mais de idade que não procuraram a polícia em decorrência do último roubo que foram vítimas, no período de referência de 365 dias, por motivo de não terem procurado a polícia em decorrência do último roubo, segundo a situação do domicílio, as': pd.DataFrame(np.random.rand(10, 6), columns=['total', 'falta de provas', 'nao era importante', 'nao acreditavam na policia', 'nao queriam envolver a policia ou medo de represalias', 'outro'])
    }
}

# Configurações de estilo
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 7)
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['axes.labelsize'] = 12

# Dicionário para guardar os caminhos das imagens geradas
image_paths = {}

# --- PREPARAÇÃO DOS DADOS DEMOGRÁFICOS ---
# Esta etapa agora prepara os dataframes separados que serão usados nos gráficos seguintes.
planilhas_demo = {
    'Agressão': dados_carregados['agressao']['Coeficientes de variação das estimativas de pessoas de 10 anos ou mais de idade que foram vítimas de agressão física, no período de referência de 365 dias, por sexo e cor ou raça, segundo a situação do domicílio, as Grandes Regiões e as Unidades da Federação'],
    'Furto': dados_carregados['furto']['Coeficientes de variação das estimativas de percentual de pessoas que foram vítimas de furto, no período de referência de 365 dias, na população de 10 anos ou mais de idade, por sexo e cor ou raça, segundo a situação do domicílio, as Grandes Regiões e as Unidades da Federação'],
    'Roubo': dados_carregados['roubo']['Coeficientes de variação das estimativas de percentual de pessoas que foram vítimas de roubo, no período de referência de 365 dias, na população de 10 anos ou mais de idade, por sexo e cor ou raça, segundo a situação do domicílio, as Grandes Regiões e as Unidades da Federação']
}
dados_grafico_sexo = {}
dados_grafico_raca = {}

for crime, df in planilhas_demo.items():
    df.columns = [c.strip().lower() for c in df.columns]
    # Separa os dados de sexo
    dados_grafico_sexo[crime] = df[['homens', 'mulheres']].mean()
    # Separa os dados de raça
    dados_grafico_raca[crime] = df[['branca', 'preta/parda']].mean()

df_sexo_comp = pd.DataFrame(dados_grafico_sexo)
df_raca_comp = pd.DataFrame(dados_grafico_raca)


# --- GRÁFICO 1A: PERFIL DEMOGRÁFICO POR SEXO ---
print("Gerando Gráfico 1A (Sexo)...")
df_sexo_comp.plot(kind='bar', figsize=(14, 8), colormap='viridis')
plt.title('Perfil Demográfico da Vítima por Sexo e Tipo de Crime\n(Coeficiente de Variação Médio)')
plt.ylabel('Coeficiente de Variação Médio')
plt.xlabel('Sexo')
plt.xticks(rotation=0)
plt.legend(title='Tipo de Crime')
plt.tight_layout()
image_paths['g1a'] = os.path.join(output_folder, '01a_perfil_demografico_sexo.png')
plt.savefig(image_paths['g1a'], bbox_inches='tight')
plt.close()

# --- GRÁFICO 1B: PERFIL DEMOGRÁFICO POR COR/RAÇA ---
print("Gerando Gráfico 1B (Cor/Raça)...")
df_raca_comp.plot(kind='bar', figsize=(14, 8), colormap='plasma')
plt.title('Perfil Demográfico da Vítima por Cor/Raça e Tipo de Crime\n(Coeficiente de Variação Médio)')
plt.ylabel('Coeficiente de Variação Médio')
plt.xlabel('Cor/Raça')
plt.xticks(rotation=0)
plt.legend(title='Tipo de Crime')
plt.tight_layout()
image_paths['g1b'] = os.path.join(output_folder, '01b_perfil_demografico_raca.png')
plt.savefig(image_paths['g1b'], bbox_inches='tight')
plt.close()


# --- GRÁFICO 2: PERFIL ETÁRIO (Sem alterações) ---
print("Gerando Gráfico 2...")
planilhas_idade = {
    'Agressão': dados_carregados['agressao']['Coeficientes de variação das estimativas de pessoas de 10 anos ou mais de idade que foram vítimas de agressão física, no período de referência de 365 dias, por grupos de idade, segundo a situação do domicílio e as Grandes Regiões'],
    'Furto': dados_carregados['furto']['Coeficientes de variação das estimativas de pessoas de 10 anos ou mais de idade que foram vítimas de furto, no período de referência de 365 dias, por grupos de idade, segundo a situação do domicílio, as Grandes Regiões e as Unidades da Federação'],
    'Roubo': dados_carregados['roubo']['Coeficientes de variação das estimativas de pessoas de 10 anos ou mais de idade que foram vítimas de roubo, no período de referência de 365 dias, por grupos de idade, segundo a situação do domicílio, as Grandes Regiões e as Unidades da Federação']
}
dados_grafico_idade = {}
for crime, df in planilhas_idade.items():
    df_sem_total = df.drop(columns=[col for col in df.columns if 'total' in col.lower()])
    dados_grafico_idade[crime] = df_sem_total.mean()
df_idade_comp = pd.DataFrame(dados_grafico_idade)
df_idade_comp.plot(kind='line', style='-o', ms=8, figsize=(14, 8))
plt.title('Perfil Etário da Vítima por Tipo de Crime\n(Coeficiente de Variação Médio)')
plt.ylabel('Coeficiente de Variação Médio')
plt.xlabel('Faixa Etária')
plt.xticks(rotation=45, ha='right')
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.legend(title='Tipo de Crime')
plt.tight_layout()
image_paths['g2'] = os.path.join(output_folder, '02_perfil_etario.png')
plt.savefig(image_paths['g2'], bbox_inches='tight')
plt.close()

# --- GRÁFICO 3: MOTIVOS NÃO REGISTRO (Sem alterações) ---
print("Gerando Gráfico 3...")
planilhas_motivos = {
    'Agressão': dados_carregados['agressao'].get('Coeficientes de variação das estimativas de pessoas de 10 anos ou mais de idade que não procuraram a polícia em decorrência da última agressão física que foram vítimas, no período de referência de 365 dias, por motivo de não terem procurado a polícia em decorrência da última agressão física, segundo algumas Unidades da Federação e Regiões Metropolitanas'),
    'Furto': dados_carregados['furto'].get('Coeficientes de variação das estimativas de pessoas de 10 anos ou mais de idade que não procuraram a polícia em decorrência do último furto que foram vítimas, no período de referência de 365 dias, por motivo de não terem procurado a polícia em decorrência do último furto, segundo a situação do domicílio,'),
    'Roubo': dados_carregados['roubo'].get('Coeficientes de variação das estimativas de pessoas de 10 anos ou mais de idade que não procuraram a polícia em decorrência do último roubo que foram vítimas, no período de referência de 365 dias, por motivo de não terem procurado a polícia em decorrência do último roubo, segundo a situação do domicílio, as')
}
dados_grafico_motivos = {}
for crime, df in planilhas_motivos.items():
    if df is not None:
        df_limp = df.drop(columns=[col for col in df.columns if 'total' in col.lower()])
        dados_grafico_motivos[crime] = df_limp.mean(numeric_only=True)
df_motivos = pd.DataFrame(dados_grafico_motivos).T
df_motivos_100 = df_motivos.div(df_motivos.sum(axis=1), axis=0) * 100
if not df_motivos_100.empty:
    df_motivos_100 = df_motivos_100.sort_values(by=list(df_motivos_100.columns), ascending=False)
fig, ax = plt.subplots(figsize=(14, 7))
cmap = plt.get_cmap('tab20c')
colors = cmap.colors[:df_motivos_100.shape[1]]
df_motivos_100.plot(kind='barh', stacked=True, ax=ax, color=colors, edgecolor='black')
ax.set_title('Motivos para Não Registrar Ocorrência (%) por Tipo de Crime', fontsize=16, weight='bold')
ax.set_xlabel('Percentual (%)', fontsize=12)
ax.set_ylabel('Tipo de Crime', fontsize=12)
ax.set_xlim(0, 100)
ax.legend(title='Motivo', bbox_to_anchor=(1.02, 1), loc='upper left', fontsize=10)
for container in ax.containers:
    ax.bar_label(container, fmt='%.1f%%', label_type='center', fontsize=9, color='white', weight='bold')
plt.tight_layout()
image_paths['g3'] = os.path.join(output_folder, '03_motivos_nao_registro.png')
plt.savefig(image_paths['g3'], bbox_inches='tight')
plt.close()

# --- GRÁFICO 4A: HEATMAP COMPARATIVO POR SEXO ---
print("Gerando Gráfico 4A (Sexo)...")
df_heatmap_sexo = df_sexo_comp.T
plt.figure(figsize=(8, 5))
sns.heatmap(df_heatmap_sexo, annot=True, fmt=".2f", cmap="YlGnBu", linewidths=.5)
plt.title('Heatmap Comparativo por Sexo e Crime')
plt.xlabel('Perfil por Sexo')
plt.ylabel('Tipo de Crime')
plt.xticks(rotation=0)
plt.yticks(rotation=0)
plt.tight_layout()
image_paths['g4a'] = os.path.join(output_folder, '04a_heatmap_comparativo_sexo.png')
plt.savefig(image_paths['g4a'], bbox_inches='tight')
plt.close()

# --- GRÁFICO 4B: HEATMAP COMPARATIVO POR RAÇA ---
print("Gerando Gráfico 4B (Raça)...")
df_heatmap_raca = df_raca_comp.T
plt.figure(figsize=(8, 5))
sns.heatmap(df_heatmap_raca, annot=True, fmt=".2f", cmap="YlOrRd", linewidths=.5)
plt.title('Heatmap Comparativo por Cor/Raça e Crime')
plt.xlabel('Perfil por Cor/Raça')
plt.ylabel('Tipo de Crime')
plt.xticks(rotation=0)
plt.yticks(rotation=0)
plt.tight_layout()
image_paths['g4b'] = os.path.join(output_folder, '04b_heatmap_comparativo_raca.png')
plt.savefig(image_paths['g4b'], bbox_inches='tight')
plt.close()


# --- GRÁFICO 5: HEATMAP CORRELAÇÃO (Sem alterações) ---
print("Gerando Gráfico 5...")
df_agressao_demo = dados_carregados['agressao']['Coeficientes de variação das estimativas de pessoas de 10 anos ou mais de idade que foram vítimas de agressão física, no período de referência de 365 dias, por sexo e cor ou raça, segundo a situação do domicílio, as Grandes Regiões e as Unidades da Federação']
df_agressao_demo.columns = [c.strip().lower() for c in df_agressao_demo.columns]
df_agressao_demo = df_agressao_demo.drop(columns=['total'])
corr_matrix = df_agressao_demo.corr()
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=1)
plt.title('Matriz de Correlação entre Perfis de Vítimas de Agressão')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
image_paths['g5'] = os.path.join(output_folder, '05_heatmap_correlacao_agressao.png')
plt.savefig(image_paths['g5'], bbox_inches='tight')
plt.close()

# --- GRÁFICO 6: BOXPLOT (Sem alterações) ---
print("Gerando Gráfico 6...")
df_agressao = dados_carregados['agressao']['Coeficientes de variação das estimativas de pessoas de 10 anos ou mais de idade que foram vítimas de agressão física, no período de referência de 365 dias, por sexo e cor ou raça, segundo a situação do domicílio, as Grandes Regiões e as Unidades da Federação']
df_agressao.columns = [c.strip().lower() for c in df_agressao.columns]
df_furto = dados_carregados['furto']['Coeficientes de variação das estimativas de percentual de pessoas que foram vítimas de furto, no período de referência de 365 dias, na população de 10 anos ou mais de idade, por sexo e cor ou raça, segundo a situação do domicílio, as Grandes Regiões e as Unidades da Federação']
df_furto.columns = [c.strip().lower() for c in df_furto.columns]
df_roubo = dados_carregados['roubo']['Coeficientes de variação das estimativas de percentual de pessoas que foram vítimas de roubo, no período de referência de 365 dias, na população de 10 anos ou mais de idade, por sexo e cor ou raça, segundo a situação do domicílio, as Grandes Regiões e as Unidades da Federação']
df_roubo.columns = [c.strip().lower() for c in df_roubo.columns]
dados_dist = {
    'Agressão': df_agressao['total'],
    'Furto': df_furto['total'],
    'Roubo': df_roubo['total']
}
df_dist = pd.DataFrame(dados_dist)
plt.figure(figsize=(10, 7))
sns.boxplot(data=df_dist, palette='viridis')
sns.stripplot(data=df_dist, color=".25", size=5)
plt.title('Distribuição do Coeficiente de Variação Total por Tipo de Crime')
plt.ylabel('Coeficiente de Variação')
plt.xlabel('Tipo de Crime')
plt.tight_layout()
image_paths['g6'] = os.path.join(output_folder, '06_boxplot_distribuicao.png')
plt.savefig(image_paths['g6'], bbox_inches='tight')
plt.close()


# --- GRÁFICO 7C: RADAR INTERSECCIONAL SEXO + RAÇA ---
print("Gerando Gráfico 7C (Sexo + Cor/Raça)...")

# Combina os DataFrames de sexo e raça em um único DataFrame interseccional
df_intersec = pd.DataFrame({
    "Homem Branco": df_sexo_comp.loc['homens'] * df_raca_comp.loc['branca'],
    "Homem Preto/Pardo": df_sexo_comp.loc['homens'] * df_raca_comp.loc['preta/parda'],
    "Mulher Branca": df_sexo_comp.loc['mulheres'] * df_raca_comp.loc['branca'],
    "Mulher Preta/Parda": df_sexo_comp.loc['mulheres'] * df_raca_comp.loc['preta/parda']
})

# Normaliza para manter os valores em escala comparável (raiz do produto)
df_intersec = df_intersec.pow(1/2)

categories_intersec = df_intersec.index.tolist()
N = len(categories_intersec)
angles = [n / float(N) * 2 * pi for n in range(N)]
angles += angles[:1]

fig, ax = plt.subplots(figsize=(9, 9), subplot_kw=dict(polar=True))
ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories_intersec, fontsize=10)
ax.set_rlabel_position(30)
plt.yticks([5, 10, 15, 20], color="grey", size=9)
plt.ylim(0, df_intersec.max().max() + 5)

cores_intersec = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
for i, perfil in enumerate(df_intersec.columns):
    values = df_intersec[perfil].tolist()
    values += values[:1]
    ax.plot(angles, values, linewidth=2, linestyle='solid', label=perfil, color=cores_intersec[i])
    ax.fill(angles, values, alpha=0.1, color=cores_intersec[i])

plt.title('Comparativo Interseccional de Perfis de Vítimas\npor Sexo e Cor/Raça (Radar)', size=15, y=1.1)
plt.legend(loc='upper right', bbox_to_anchor=(0.15, 0.1))
plt.tight_layout()

image_paths['g7c'] = os.path.join(output_folder, '07c_radar_interseccional.png')
plt.savefig(image_paths['g7c'], bbox_inches='tight')
plt.close()


# --- GRÁFICO 8A: RANKING POR SEXO ---
print("Gerando Gráfico 8A (Sexo)...")
ranking_sexo = df_sexo_comp.mean(axis=1).sort_values(ascending=False)
plt.figure(figsize=(10, 4))
sns.barplot(x=ranking_sexo.values, y=ranking_sexo.index, palette='rocket', orient='h')
plt.title('Ranking de Perfis por Sexo (Todos os Crimes)')
plt.xlabel('Coeficiente de Variação Médio Agregado')
plt.ylabel('Sexo')
for index, value in enumerate(ranking_sexo):
    plt.text(value, index, f' {value:.2f}', va='center')
plt.tight_layout()
image_paths['g8a'] = os.path.join(output_folder, '08a_ranking_sexo.png')
plt.savefig(image_paths['g8a'], bbox_inches='tight')
plt.close()

# --- GRÁFICO 8B: RANKING POR RAÇA ---
print("Gerando Gráfico 8B (Raça)...")
ranking_raca = df_raca_comp.mean(axis=1).sort_values(ascending=False)
plt.figure(figsize=(10, 4))
sns.barplot(x=ranking_raca.values, y=ranking_raca.index, palette='mako', orient='h')
plt.title('Ranking de Perfis por Cor/Raça (Todos os Crimes)')
plt.xlabel('Coeficiente de Variação Médio Agregado')
plt.ylabel('Cor/Raça')
for index, value in enumerate(ranking_raca):
    plt.text(value, index, f' {value:.2f}', va='center')
plt.tight_layout()
image_paths['g8b'] = os.path.join(output_folder, '08b_ranking_raca.png')
plt.savefig(image_paths['g8b'], bbox_inches='tight')
plt.close()


# --- GRÁFICO 9A: DUMBBELL SEXO ---
print("Gerando Gráfico 9A (Sexo)...")
# O dataframe de entrada agora é o df_sexo_comp
df_dumbbell_sexo = df_sexo_comp.T.sort_values(by='homens')
fig, ax = plt.subplots(figsize=(10, 6))
ax.hlines(y=df_dumbbell_sexo.index, xmin=df_dumbbell_sexo['homens'], xmax=df_dumbbell_sexo['mulheres'], color='gray', alpha=0.7, lw=2)
ax.scatter(df_dumbbell_sexo['homens'], df_dumbbell_sexo.index, color='#1f77b4', s=100, label='Homens')
ax.scatter(df_dumbbell_sexo['mulheres'], df_dumbbell_sexo.index, color='#ff7f0e', s=100, label='Mulheres')
ax.set_title('Comparativo do Coeficiente de Variação: Homens vs. Mulheres')
ax.set_xlabel('Coeficiente de Variação Médio')
ax.set_ylabel('Tipo de Crime')
ax.legend()
plt.tight_layout()
image_paths['g9a'] = os.path.join(output_folder, '09a_dumbbell_homens_mulheres.png')
plt.savefig(image_paths['g9a'], bbox_inches='tight')
plt.close()

# --- GRÁFICO 9B: DUMBBELL RAÇA (NOVO) ---
print("Gerando Gráfico 9B (Raça)...")
df_dumbbell_raca = df_raca_comp.T.sort_values(by='branca')
fig, ax = plt.subplots(figsize=(10, 6))
ax.hlines(y=df_dumbbell_raca.index, xmin=df_dumbbell_raca['branca'], xmax=df_dumbbell_raca['preta/parda'], color='gray', alpha=0.7, lw=2)
ax.scatter(df_dumbbell_raca['branca'], df_dumbbell_raca.index, color='#d62728', s=100, label='Branca')
ax.scatter(df_dumbbell_raca['preta/parda'], df_dumbbell_raca.index, color='#9467bd', s=100, label='Preta/Parda')
ax.set_title('Comparativo do Coeficiente de Variação: Branca vs. Preta/Parda')
ax.set_xlabel('Coeficiente de Variação Médio')
ax.set_ylabel('Tipo de Crime')
ax.legend()
plt.tight_layout()
image_paths['g9b'] = os.path.join(output_folder, '09b_dumbbell_branca_parda.png')
plt.savefig(image_paths['g9b'], bbox_inches='tight')
plt.close()


# --- GRÁFICO 10: FACET GRID (Sem alterações, já separa bem as categorias) ---
print("Gerando Gráfico 10...")
df_agressao_geo = dados_carregados['agressao']['Coeficientes de variação das estimativas de pessoas de 10 anos ou mais de idade que foram vítimas de agressão física, no período de referência de 365 dias, por sexo e cor ou raça, segundo a situação do domicílio, as Grandes Regiões e as Unidades da Federação']
df_agressao_geo.columns = [c.strip().lower() for c in df_agressao_geo.columns]
df_agressao_geo = df_agressao_geo.drop(columns=['total'])
df_agressao_geo['UF'] = [f'UF_{i+1}' for i in range(len(df_agressao_geo))]
df_tidy = df_agressao_geo.melt(id_vars='UF', var_name='Perfil', value_name='Coeficiente')
g = sns.FacetGrid(df_tidy, col="UF", col_wrap=4, height=3, sharey=False)
g.map(sns.barplot, "Perfil", "Coeficiente", palette='plasma', order=['homens', 'mulheres', 'branca', 'preta/parda'])
g.fig.suptitle('Perfil de Vítimas de Agressão por UF', y=1.03, fontsize=16)
g.set_axis_labels("Perfil Demográfico", "Coeficiente de Variação")
g.set_titles("UF: {col_name}")
for ax in g.axes.flatten():
    for label in ax.get_xticklabels():
        label.set_rotation(45)
g.fig.tight_layout()
image_paths['g10'] = os.path.join(output_folder, '10_facetgrid_agressao_uf.png')
g.savefig(image_paths['g10'])
plt.close()

print("\n--- Análise concluída e todos os gráficos foram salvos na pasta 'graficos' ---")


--- Iniciando geração sequencial de gráficos ---
Pasta 'graficos' pronta para receber os gráficos.
Gerando Gráfico 1A (Sexo)...
Gerando Gráfico 1B (Cor/Raça)...
Gerando Gráfico 2...
Gerando Gráfico 3...
Gerando Gráfico 4A (Sexo)...
Gerando Gráfico 4B (Raça)...
Gerando Gráfico 5...
Gerando Gráfico 6...
Gerando Gráfico 7C (Sexo + Cor/Raça)...
Gerando Gráfico 8A (Sexo)...


C:\Users\pedro\AppData\Local\Temp\ipykernel_12188\659801336.py:261: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=ranking_sexo.values, y=ranking_sexo.index, palette='rocket', orient='h')
C:\Users\pedro\AppData\Local\Temp\ipykernel_12188\659801336.py:276: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=ranking_raca.values, y=ranking_raca.index, palette='mako', orient='h')


Gerando Gráfico 8B (Raça)...
Gerando Gráfico 9A (Sexo)...
Gerando Gráfico 9B (Raça)...
Gerando Gráfico 10...


c:\Users\pedro\AppData\Local\Programs\Python\Python313\Lib\site-packages\seaborn\axisgrid.py:854: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  func(*plot_args, **plot_kwargs)
c:\Users\pedro\AppData\Local\Programs\Python\Python313\Lib\site-packages\seaborn\axisgrid.py:854: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  func(*plot_args, **plot_kwargs)
c:\Users\pedro\AppData\Local\Programs\Python\Python313\Lib\site-packages\seaborn\axisgrid.py:854: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  func(*plot_args, **plot_kwargs)
c:\Users\pedro\AppData\Local\Programs\Python\Python313\


--- Análise concluída e todos os gráficos foram salvos na pasta 'graficos' ---


In [28]:
print("\n--- Preparando textos descritivos e análises para o relatório ---")

analises = {
    "Gráfico 1A: Perfil por Sexo (Barras)": {
        "objetivo": "Comparar a variabilidade (inconstância) das estimativas de vitimização entre homens e mulheres para cada tipo de crime principal.",
        "o_que_mostra": "O gráfico exibe barras agrupadas por tipo de crime. Dentro de cada grupo, compara-se o Coeficiente de Variação (CV) médio para 'homens' e 'mulheres'. Barras mais altas indicam maior instabilidade e dispersão nos dados para aquele grupo.",
        "analise": "A análise deste gráfico permite identificar se há um sexo cuja vitimização é estimada com menos consistência. Por exemplo, pode-se observar se o CV para mulheres vítimas de agressão é maior que para homens, sugerindo uma variação regional mais acentuada nos dados desse grupo, o que exige cautela ao generalizar os dados nacionais.",
        "imagem": image_paths.get('g1a')
    },
    "Gráfico 1B: Perfil por Cor/Raça (Barras)": {
        "objetivo": "Comparar a variabilidade das estimativas de vitimização entre os perfis de cor/raça ('branca' e 'preta/parda') para cada tipo de crime.",
        "o_que_mostra": "Similar ao gráfico anterior, este agrupa por crime e compara o CV médio para os grupos 'branca' e 'preta/parda'. A altura da barra representa a instabilidade da estimativa.",
        "analise": "Este gráfico é crucial para analisar disparidades raciais na consistência dos dados. É comum observar um CV sistematicamente mais elevado para a população 'preta/parda', indicando que as estimativas de vitimização para este grupo são mais heterogêneas entre as localidades, o que pode mascarar realidades locais distintas.",
        "imagem": image_paths.get('g1b')
    },
    "Gráfico 2: Perfil por Faixa Etária (Linhas)": {
        "objetivo": "Analisar como a variabilidade das estimativas de vitimização se comporta através das diferentes faixas etárias para cada tipo de crime.",
        "o_que_mostra": "O gráfico apresenta a tendência da variabilidade (CV médio) ao longo de várias faixas etárias. Cada linha representa um tipo de crime. Picos na linha indicam maior instabilidade para aquela faixa etária.",
        "analise": "As linhas revelam picos de instabilidade em idades específicas. Frequentemente, a faixa de '16 a 24 anos' exibe o maior CV, especialmente para roubos, refletindo maior variação geográfica na vitimização desse grupo. A tendência pode mostrar se a confiabilidade dos dados aumenta ou diminui com a idade.",
        "imagem": image_paths.get('g2')
    },
    "Gráfico 3: Motivos para Não Registrar Ocorrência (Barras Empilhadas)": {
        "objetivo": "Revelar e comparar a distribuição percentual dos motivos que levaram as vítimas a não procurarem a polícia, distinguindo por tipo de crime.",
        "o_que_mostra": "Cada barra horizontal (100%) representa um tipo de crime. Os segmentos coloridos mostram a proporção de cada motivo. O tamanho do segmento indica sua importância relativa.",
        "analise": "A análise aponta para diferentes barreiras de acesso à justiça. Para furtos, o motivo 'não era importante' tende a dominar, sugerindo baixa percepção de dano. Para agressões, 'medo de represálias' ou 'não acreditavam na polícia' podem ter maior peso, indicando uma falha de confiança ou segurança no sistema.",
        "imagem": image_paths.get('g3')
    },
    "Gráfico 4A: Heatmap Comparativo por Sexo": {
        "objetivo": "Fornecer uma visualização rápida da intensidade da variabilidade dos dados para cada combinação de tipo de crime e sexo.",
        "o_que_mostra": "Uma matriz onde linhas são os crimes e colunas são os sexos. A cor de cada célula indica a magnitude do CV, com cores mais intensas significando maior variabilidade. O valor numérico exato está anotado na célula.",
        "analise": "O heatmap permite identificar rapidamente os 'pontos quentes'. Por exemplo, a célula 'Agressão-Mulheres' pode ter a cor mais intensa, indicando que esta é a combinação com a estimativa mais instável em todo o conjunto de dados, sendo um ponto crítico para análise.",
        "imagem": image_paths.get('g4a')
    },
    "Gráfico 4B: Heatmap Comparativo por Cor/Raça": {
        "objetivo": "Fornecer uma visualização rápida da intensidade da variabilidade dos dados para cada combinação de tipo de crime e cor/raça.",
        "o_que_mostra": "Uma matriz onde linhas são os crimes e colunas são os grupos de cor/raça. A cor de cada célula indica a magnitude do CV, com cores mais intensas significando maior variabilidade.",
        "analise": "Este heatmap expõe qual crime apresenta maior instabilidade de dados para qual grupo racial. Uma célula como 'Roubo - Preta/Parda' com valor elevado sugere que as médias nacionais para este cruzamento devem ser interpretadas com extrema cautela, pois não representam bem as diversas realidades regionais.",
        "imagem": image_paths.get('g4b')
    },
    "Gráfico 5: Heatmap de Correlação (Crime de Agressão)": {
        "objetivo": "Investigar se a variabilidade nas estimativas de vitimização por agressão dos diferentes perfis demográficos se movem em conjunto.",
        "o_que_mostra": "A matriz exibe o coeficiente de correlação (de -1 a 1) entre os CVs de todos os pares de perfis. Cores quentes (próximas de 1) indicam correlação positiva; cores frias (próximas de -1) indicam correlação negativa.",
        "analise": "Uma correlação positiva forte (ex: > 0.7) entre 'homens' e 'mulheres' significa que, nas regiões onde a estimativa para homens é instável, a para mulheres também tende a ser. Isso pode sugerir que fatores geográficos, e não apenas o perfil, causam a instabilidade dos dados de agressão.",
        "imagem": image_paths.get('g5')
    },
    "Gráfico 6: Distribuição com Boxplot": {
        "objetivo": "Comparar a distribuição completa (mediana, quartis, outliers) do CV 'Total' entre os tipos de crime.",
        "o_que_mostra": "Cada 'caixa' representa um tipo de crime. A linha na caixa é a mediana; a altura da caixa é a dispersão dos 50% centrais dos dados; as hastes indicam o alcance geral e pontos isolados são outliers.",
        "analise": "O boxplot vai além da média. Pode revelar que, embora 'Roubo' tenha uma média de CV similar a 'Furto', sua 'caixa' é muito mais alta, indicando maior inconsistência e dispersão nos dados. Outliers apontam para UFs com variabilidade excepcionalmente alta.",
        "imagem": image_paths.get('g6')
    },
    "Gráfico 7C: Radar Comparativo por Sexo + Cor/Raça": {
        "objetivo": "Unificar a análise visual da variabilidade das estimativas de vitimização combinando os fatores de sexo e cor/raça, permitindo observar perfis interseccionais.",
        "o_que_mostra": (
            "O gráfico exibe um radar com quatro perfis combinados: 'Homem Branco', 'Homem Preto/Pardo', "
            "'Mulher Branca' e 'Mulher Preta/Parda'. Para cada tipo de crime, é desenhado um polígono cuja forma reflete a distribuição da variabilidade das estimativas entre esses perfis. Perfis com maior distância do centro apresentam maior instabilidade."
        ),
        "analise": (
            "Esse radar interseccional permite identificar, de forma integrada, quais perfis demográficos estão mais sujeitos à instabilidade nas estatísticas de vitimização. Por exemplo, se o polígono de 'Agressão' se estende mais no eixo 'Mulher Preta/Parda', isso indica que as estimativas para esse perfil são mais variáveis, destacando a necessidade de abordagens políticas e analíticas interseccionais."
        ),
        "imagem": image_paths.get('g7c')
    },
    "Gráfico 8A: Ranking de Perfis por Sexo": {
        "objetivo": "Classificar os perfis de sexo com base na sua variabilidade de dados média, agregando os resultados de todos os crimes.",
        "o_que_mostra": "Um gráfico de barras horizontais simples onde a barra mais longa pertence ao perfil de sexo cuja estimativa de vitimização é, na média geral, a mais instável.",
        "analise": "Este gráfico fornece uma conclusão direta sobre qual sexo tem os dados mais heterogêneos no geral. A diferença no comprimento das barras quantifica essa disparidade na confiabilidade média das estimativas.",
        "imagem": image_paths.get('g8a')
    },
    "Gráfico 8B: Ranking de Perfis por Cor/Raça": {
        "objetivo": "Classificar os perfis de cor/raça com base na sua variabilidade de dados média, agregando os resultados de todos os crimes.",
        "o_que_mostra": "Um gráfico de barras horizontais simples onde a barra mais longa pertence ao perfil de cor/raça cuja estimativa de vitimização é, na média geral, a mais instável.",
        "analise": "O ranking agregado por raça geralmente evidencia a maior vulnerabilidade estatística da população 'preta/parda'. A barra significativamente maior para este grupo indica que as políticas públicas baseadas em dados agregados podem não atender adequadamente às suas necessidades específicas.",
        "imagem": image_paths.get('g8b')
    },
    "Gráfico 9A: Comparativo Homens vs. Mulheres (Dumbbell)": {
        "objetivo": "Isolar e comparar diretamente a diferença na variabilidade das estimativas entre homens e mulheres para cada tipo de crime.",
        "o_que_mostra": "Para cada crime, dois pontos são plotados (um para homens, outro para mulheres). Uma linha conecta os pontos, destacando a magnitude da diferença ('gap') entre os sexos.",
        "analise": "O dumbbell plot é excelente para visualizar o 'gap' de instabilidade. O crime com a linha de conexão mais longa indica uma diferença muito acentuada na confiabilidade dos dados entre homens e mulheres, sinalizando a necessidade de análises aprofundadas.",
        "imagem": image_paths.get('g9a')
    },
    "Gráfico 9B: Comparativo Branca vs. Preta/Parda (Dumbbell)": {
        "objetivo": "Isolar e comparar diretamente a diferença na variabilidade das estimativas entre os grupos de cor/raça para cada tipo de crime.",
        "o_que_mostra": "Para cada crime, dois pontos são plotados (um para 'branca', outro para 'preta/parda'). Uma linha conecta os pontos, evidenciando a disparidade racial na consistência dos dados.",
        "analise": "Este gráfico expõe a desigualdade na precisão dos dados. Se o ponto 'preta/parda' estiver consistentemente à direita do ponto 'branca', isso indica uma fragilidade estrutural na coleta ou na homogeneidade dos dados para esse grupo.",
        "imagem": image_paths.get('g9b')
    },
    "Gráfico 10: Variação Geográfica (Facet Grid)": {
        "objetivo": "Desagregar a análise nacional para investigar como o perfil de variabilidade das vítimas de agressão se comporta em cada Unidade da Federação (UF).",
        "o_que_mostra": "Uma grade de múltiplos mini-gráficos de barras. Cada mini-gráfico representa uma UF, mostrando os CVs para cada perfil. Permite comparar os padrões locais com a média nacional.",
        "analise": "A análise por UF é fundamental para a ação local. Pode-se identificar que, embora a média nacional do CV para 'homens' seja baixa, em uma UF específica ela seja a mais alta, indicando que as estratégias de segurança e de coleta de dados devem ser regionalizadas.",
        "imagem": image_paths.get('g10')
    }
}

analise_geral = {
    "titulo": "Análise Geral Consolidada",
    "texto": (
        "A análise consolidada dos gráficos revela uma narrativa complexa sobre a estabilidade das estimativas de vitimização no Brasil. "
        "O Coeficiente de Variação (CV) indica o grau de imprecisão dos dados, servindo como um importante indicador para a confiabilidade das estatísticas de criminalidade.\n\n"
        "A separação da análise por sexo e cor/raça, evidenciada em diversos gráficos (como os Gráficos 1A/B, 4A/B, 8B), deixa claro que a variabilidade não é uniforme. "
        "Perfis demográficos específicos, por exemplo, apresentam consistentemente CVs mais elevados, o que pode mascarar realidades locais diversas. \n\n"
        "Destaca-se o Gráfico 7C, que, ao unir as dimensões de sexo e cor/raça em uma análise interseccional, evidencia que perfis como 'Mulher Preta/Parda' podem apresentar maior instabilidade nas estatísticas em vários tipos de crime. "
        "Isso reforça a necessidade de abordagens políticas e metodológicas que considerem as intersecções dos marcadores sociais para uma análise mais precisa.\n\n"
        "Em suma, os dados sobre vitimização no Brasil demonstram uma heterogeneidade significativa, onde a confiabilidade das estimativas depende do perfil da vítima, do tipo de crime e da localidade. "
        "Políticas públicas e metodologias de coleta de dados devem ser sensíveis a essas múltiplas dimensões para efetivamente combater as desigualdades e promover uma análise realista dos fenômenos criminais."
    )
}

# ---------------------------------------------------------------------------------
# Parte 3: Geração do PDF Atualizado
# ---------------------------------------------------------------------------------
class PDF(FPDF):
    def header(self):
        self.set_font('Helvetica', 'B', 15)
        self.set_text_color(40, 40, 40)
        self.cell(0, 8, 'Relatório de Análise de Criminalidade no Brasil', ln=1, align='C')
        self.set_font('Helvetica', '', 11)
        self.set_text_color(100, 100, 100)
        self.cell(0, 6, 'Estudo da variabilidade nas estimativas de vitimização', ln=1, align='C')
        self.ln(5)
        # Linha horizontal fina
        self.set_draw_color(180, 180, 180)
        self.set_line_width(0.3)
        self.line(self.l_margin, self.get_y(), self.w - self.r_margin, self.get_y())
        self.ln(5)

    def footer(self):
        self.set_y(-15)
        self.set_font('Helvetica', 'I', 8)
        self.set_text_color(120, 120, 120)
        data = datetime.now().strftime("%d/%m/%Y")
        self.cell(0, 10, f'Página {self.page_no()}  |  Emitido em: {data}', 0, 0, 'C')

    def chapter_title(self, title):
        self.set_font('Helvetica', 'B', 13)
        self.set_text_color(0)
        self.cell(0, 10, title, ln=1)
        self.set_draw_color(0, 0, 0)
        self.set_line_width(0.2)
        self.line(self.l_margin, self.get_y(), self.w - self.r_margin, self.get_y())
        self.ln(4)

    def chapter_body(self, body_dict):
        def add_section(title, text):
            self.set_font('Helvetica', 'B', 12)
            self.set_text_color(30, 30, 30)
            self.cell(0, 7, title, ln=1)
            self.set_font('Helvetica', '', 12)
            self.set_text_color(50, 50, 50)
            self.multi_cell(0, 6, text)
            self.ln(2)

        add_section("Objetivo:", body_dict.get("objetivo", ""))
        add_section("O que o Gráfico Mostra:", body_dict.get("o_que_mostra", ""))
        add_section("Análise do Gráfico:", body_dict.get("analise", ""))

        # Adiciona imagem (se houver)
        if body_dict.get("imagem") and os.path.exists(body_dict["imagem"]):
            self.ln(1)
            img_width = self.w - 2 * self.l_margin
            self.image(body_dict["imagem"], x=self.l_margin, w=img_width)
        else:
            self.set_font('Helvetica', 'I', 11)
            self.set_text_color(100, 0, 0)
            self.cell(0, 10, "[Imagem não encontrada ou caminho inválido]", ln=1)
        self.ln(6)

pdf = PDF('P', 'mm', 'A4')
pdf.set_auto_page_break(auto=True, margin=20)

# Adiciona uma página para cada gráfico
for titulo, conteudo in analises.items():
    pdf.add_page()
    pdf.chapter_title(titulo)
    pdf.chapter_body(conteudo)

# Página final com a análise geral consolidada
pdf.add_page()
pdf.chapter_title(analise_geral['titulo'])
pdf.set_font('Helvetica', '', 12)
pdf.set_text_color(40, 40, 40)
pdf.multi_cell(0, 6, analise_geral['texto'])

# Exporta o PDF
pdf_path = os.path.join(output_dir, 'relatorio_pnad_2009.pdf')
pdf.output(pdf_path)

print(f"\n✅ PDF gerado com sucesso em: {pdf_path}")


--- Preparando textos descritivos e análises para o relatório ---


C:\Users\pedro\AppData\Local\Temp\ipykernel_12188\2304274773.py:116: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0, 8, 'Relatório de Análise de Criminalidade no Brasil', ln=1, align='C')
C:\Users\pedro\AppData\Local\Temp\ipykernel_12188\2304274773.py:119: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0, 6, 'Estudo da variabilidade nas estimativas de vitimização', ln=1, align='C')
C:\Users\pedro\AppData\Local\Temp\ipykernel_12188\2304274773.py:137: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0, 10, title, ln=1)
C:\Users\pedro\AppData\Local\Temp\ipykernel_12188\2304274773.py:147: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0, 7,


✅ PDF gerado com sucesso em: ../../analises/relatorio_pnad_2009.pdf


In [29]:
import shutil

# Nome da pasta que você deseja deletar
folder_to_delete = 'graficos'

# Verifica se a pasta realmente existe
if os.path.exists(folder_to_delete):
    try:
        # Usa shutil.rmtree() para remover a pasta e todo o seu conteúdo
        shutil.rmtree(folder_to_delete)
        print(f"A pasta '{folder_to_delete}' e todo o seu conteúdo foram deletados com sucesso.")
    except OSError as e:
        print(f"Erro ao deletar a pasta '{folder_to_delete}': {e}")
else:
    print(f"A pasta '{folder_to_delete}' não foi encontrada. Nenhuma ação foi tomada.")

A pasta 'graficos' e todo o seu conteúdo foram deletados com sucesso.
